# Generating melody *and* harmony using LSTMs

Again, using data from the Weimar Jazz Database.

## Approach #1: Add underlying harmony directly to note events

Functions here will eventually be migrated to the `jazzaiexperiments` module.

Import everything we need:

In [1]:
import os

import mido
import music21
import pandas as pd

from jazzaiexperiments import *  # For easier testing/debugging

Using TensorFlow backend.


### Add chord info (from database) to note events

In [2]:
# Testing + building jazzaiexperiments functions
# Based on lstm_train_on_midi_input()

# Set variables (that will be passed in as arguments)
# tune_name = "ColemanHawkins_BodyAndSoul_FINAL"
tune_name = "JohnColtrane_GiantSteps-1_FINAL"
midi_data_dir = "../data/midi/quantized/"
checkpoints_data_dir = "../data/models/"
seq_length = 10
num_epochs = 1

# Create note events
input_filepath = midi_construct_input_filepath(tune_name, midi_data_dir)
midi_track = midi_load_melody_from_file(input_filepath)
note_pairs = midi_extract_note_pairs(midi_track)
note_pairs = midi_normalize_velocities(note_pairs, interval=10)
note_events = midi_create_note_events(note_pairs)
print("Created note events from {}".format(input_filepath))

# Format note data to feed into network
note_set = midi_create_note_set(note_events)
seqs_input, seqs_output = midi_split_subsequences(note_events,
                                                  seq_length=seq_length)
num_seqs = len(seqs_input)
seq_length = len(seqs_input[0])
num_unique_notes = len(note_set)
x, y = midi_format_for_lstm(seqs_input, seqs_output,
                            num_seqs=num_seqs,
                            seq_length=seq_length,
                            num_unique_notes=num_unique_notes)
print("Formatted note data ({} seqs of length {}, "
      "{} unique notes)".format(num_seqs, seq_length, num_unique_notes))

Created note events from ../data/midi/quantized/JohnColtrane_GiantSteps-1_FINAL.mid
Formatted note data (1117 seqs of length 10, 335 unique notes)


In [3]:
def midi_note_event_to_dict(note):
    note_events_keys = midi_get_note_event_keys()
    return dict((note_events_keys[i], note[i]) for i,_ in enumerate(note))

midi_note_event_to_dict(note_events[0])

{'noteoff_time': 96,
 'noteon_pitch': 61,
 'noteon_time': 0,
 'noteon_velocity': 110}

In [4]:
[msg for msg in midi_track if "note" not in msg.type]

[<meta message track_name name='Melody' time=0>,
 <meta message set_tempo tempo=206185 time=0>,
 <meta message key_signature key='Eb' time=0>,
 <meta message time_signature numerator=4 denominator=4 clocks_per_click=24 notated_32nd_notes_per_beat=8 time=0>,
 <meta message end_of_track time=0>]

In [5]:
midi_track[5]

<message note_off channel=0 note=61 velocity=113 time=96>

In [6]:
mf = mido.MidiFile(input_filepath)
mf.ticks_per_beat

192

In [7]:
# Calculate times of note onsets (in seconds)
def calculate_note_times_seconds(input_filepath):
    midi_file = mido.MidiFile(input_filepath)
    midi_track = midi_load_melody_from_file(input_filepath)
    tempo = midi_track[1].tempo
    # ppq = midi_track[3].clocks_per_click
    # n32 = midi_track[3].notated_32nd_notes_per_beat
    ppq = midi_file.ticks_per_beat
    note_times = [mido.tick2second(msg.time, ppq, tempo) for msg in midi_track if "note" in msg.type]
    note_times_summed = []
    for i, t in enumerate(note_times):
        note_times_summed.append(sum(note_times[:i]) + t)
    return note_times_summed

calculate_note_times_seconds(input_filepath)[:10]

[0.0,
 0.10309249999999998,
 0.10309249999999998,
 0.20618499999999995,
 0.20618499999999995,
 0.3092774999999999,
 0.3092774999999999,
 0.4123699999999999,
 0.4123699999999999,
 0.7216474999999998]

Actually, let's get the current chord directly from the database. Here I've exported the **beats** and **melody** from the database using a simple:

    SELECT * FROM beats WHERE melid=96
    
and

    SELECT * FROM melody WHERE melid=96
    
with 96 being the melody ID for Coleman Hawkins - Body and Soul

In [8]:
len(note_events)

1127

In [9]:
def db_construct_filepath(tune_name,
                          data_type,  # "beats", "melody"
                          data_dir="../data/db/"):
    filename = "{}_{}.csv".format(tune_name, data_type)
    filepath = os.path.join(data_dir, filename)
    return filepath

beats_filepath = db_construct_filepath(tune_name, "beats")
melody_filepath = db_construct_filepath(tune_name, "melody")

(beats_filepath, melody_filepath)

('../data/db/JohnColtrane_GiantSteps-1_FINAL_beats.csv',
 '../data/db/JohnColtrane_GiantSteps-1_FINAL_melody.csv')

In [10]:
def db_read_file(filepath):
    data = pd.read_csv(filepath)
    return data

db_read_file(beats_filepath).head()

,beatid,melid,onset,bar,beat,signature,chord,form,bass_pitch,chorus_id
0,61593,222,2.118821,-1,1,NaN,NC,I1,45,0
1,61594,222,2.328946,-1,2,NaN,NaN,NaN,45,0
2,61595,222,2.539070,-1,3,NaN,NaN,NaN,43,0
3,61596,222,2.746576,-1,4,NaN,NaN,NaN,43,0
4,61597,222,2.954082,0,1,NaN,NaN,NaN,43,0


In [11]:
db_read_file(melody_filepath).head()

,eventid,melid,onset,pitch,duration,period,division,bar,beat,tatum,...,f0_mod,loud_max,loud_med,loud_sd,loud_relpos,loud_cent,loud_s2b,f0_range,f0_freq_hz,f0_med_dev
0,100681,222,2.925714,61.0,0.063855,4,2,0,1,1,...,NaN,0.457511,68.801593,6.779312,0.833333,0.630457,1.018265,11.997876,NaN,-1.965011
1,100682,222,3.044717,64.0,0.084172,4,2,0,1,2,...,NaN,0.841233,70.130213,4.227650,0.750000,0.573029,1.044817,15.933515,NaN,8.342188
2,100683,222,3.143401,68.0,0.087075,4,2,0,2,1,...,NaN,0.695400,71.103658,2.422423,0.625000,0.591172,1.074080,10.007219,NaN,-15.379263
3,100684,222,3.244989,71.0,0.095782,4,2,0,2,2,...,NaN,0.894560,72.128914,2.825900,0.375000,0.567110,1.105573,12.527423,NaN,9.463251
4,100685,222,3.355283,70.0,0.269932,4,1,0,3,1,...,NaN,0.817975,71.566897,3.584426,0.400000,0.473240,1.048796,13.543748,3.744905,-13.802721


In [12]:
# data_beats.dropna?

In [13]:
def db_get_harmony_for_melody(beats_filepath, melody_filepath):
    data_beats = db_read_file(beats_filepath)
    data_melody = db_read_file(melody_filepath)
    chords = []
    for i,melevt in data_melody.iterrows():
        # Get beats that came before current melody event (i.e. note)
        beats = data_beats.dropna(subset=["chord"])[data_beats.onset < melevt.onset]
        if len(beats) < 1:
            chords.append("NC")
            continue

        # Get most recent chord
        most_recent_beat = beats.iloc[-1:]
        #print("{}: {}".format(i, len(beats)))
        #print(most_recent_beat.chord)
        chord = most_recent_beat.chord
        chord = chord[chord.keys()[0]] if len(chord.keys()) > 0 else "NC"
        chords.append(chord)
    return chords
        
chords = db_get_harmony_for_melody(beats_filepath, melody_filepath)
chords[:10]

/Users/usdivad/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


['NC', 'NC', 'NC', 'NC', 'NC', 'NC', 'Cbj7', 'Cbj7', 'Cbj7', 'D7']

In [14]:
# This will be a variant of midi_create_note_events()
def midi_create_note_events_harmony(note_pairs, chords):
    note_events = [(note_on.note, note_on.velocity,
                    note_on.time, note_off.time,
                    chords[i])
                   for i, (note_on, note_off) in enumerate(note_pairs)]
    return note_events

note_events = midi_create_note_events_harmony(note_pairs, chords)
note_events[:10]

[(61, 110, 0, 96, 'NC'),
 (64, 110, 0, 96, 'NC'),
 (68, 110, 0, 96, 'NC'),
 (71, 120, 0, 96, 'NC'),
 (70, 110, 0, 288, 'NC'),
 (68, 110, 0, 96, 'NC'),
 (66, 110, 0, 96, 'Cbj7'),
 (63, 110, 0, 96, 'Cbj7'),
 (59, 100, 0, 240, 'Cbj7'),
 (62, 110, 0, 96, 'D7')]

In [15]:
# We already did this in db_get_harmony_for_melody() and midi_create_note_events_harmony()
# def midi_add_harmony_to_notes(note_events):
#     # TODO
#     # So we'll need the note events but also the chordal info. From a CSV export from the database itself??
#     pass

Now let's put it all back into our function:

In [16]:
# def lstm_train_with_harmony():

def lstm_train_on_midi_input(tune_name,
                             midi_data_dir="../data/midi/",
                             checkpoints_data_dir="../data/models/",
                             input_filepath=None,
                             weights_filepath=None,
                             db_beats_filepath=None,
                             db_melody_filepath=None,
                             seq_length=10,
                             num_epochs=100,
                             mode="single_melody"):
    """Build and train an LSTM from an input MIDI file.
    
    To load weights, just pass in a weights_filepath. If no additional training is needed,
    just set num_epochs to 0.
    """
    # Load MIDI file
    if input_filepath is None:
        input_filepath = midi_construct_input_filepath(tune_name,
                                                       midi_data_dir)
    midi_track = midi_load_melody_from_file(input_filepath)
    note_pairs = midi_extract_note_pairs(midi_track)
    note_pairs = midi_normalize_velocities(note_pairs, interval=10)
    
    # Get harmony data
    chords = db_get_harmony_for_melody(db_beats_filepath, db_melody_filepath)
    
    # Create note events
    note_events = []
    if mode == "single_melody":
        note_events = midi_create_note_events(note_pairs)
    elif mode == "single_melody_harmony":
        note_events = midi_create_note_events_harmony(note_pairs, chords)
    print("Created {} note events from {} using mode {}".format(len(note_events), input_filepath, mode))

    # Format note data to feed into network
    note_set = midi_create_note_set(note_events)
    seqs_input, seqs_output = midi_split_subsequences(note_events,
                                                      seq_length=seq_length)
    num_seqs = len(seqs_input)
    seq_length = len(seqs_input[0])
    num_unique_notes = len(note_set)
    x, y = midi_format_for_lstm(seqs_input, seqs_output,
                                num_seqs=num_seqs,
                                seq_length=seq_length,
                                num_unique_notes=num_unique_notes)
    print("Formatted note data ({} seqs of length {}, "
          "{} unique notes)".format(num_seqs, seq_length, num_unique_notes))

    # Create LSTM
    model = lstm_create(x.shape, y.shape, num_units=256, dropout_rate=0.2)
    print("Created model")

    # Train LSTM, or load from weights
    if weights_filepath is not None:
        model = lstm_load_weights(model, weights_filepath)
        print("Loaded weights from {}".format(weights_filepath))
    else:
        print("No weights loaded (to load weights, specify a `weights_filepath`)")
        
    if num_epochs > 0:
        callbacks = lstm_setup_callbacks(tune_name, checkpoints_data_dir)
        model = lstm_fit_model(model, x, y,
                               num_epochs=num_epochs,
                               batch_size=32,
                               callbacks=callbacks)
        print("Trained model over {} epochs".format(num_epochs))
    else:
        print("No training needed (`num_epochs` specified as 0)")

    return (model, note_events, input_filepath)
    
# # Set variables (that will be passed in as arguments)
# tune_name = "ColemanHawkins_BodyAndSoul_FINAL"
# midi_data_dir = "../data/midi/quantized/"
# checkpoints_data_dir = "../data/models/"
# input_filepath = None
# beats_filepath = "../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv"
# melody_filepath = "../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv"
# weights_filepath = None
# seq_length = 10
# num_epochs = 1
# mode = "single_melody_harmony"
# #    

# TESTING
# trained = lstm_train_on_midi_input(tune_name="ColemanHawkins_BodyAndSoul_FINAL",
#                                    midi_data_dir="../data/midi/quantized/",
#                                    db_beats_filepath="../data/db/ColemanHawkins_BodyAndSoul_FINAL_beats.csv",
#                                    db_melody_filepath="../data/db/ColemanHawkins_BodyAndSoul_FINAL_melody.csv",
#                                    seq_length=10,
#                                    num_epochs=1,
#                                    mode="single_melody_harmony")
# model, note_events, input_filepath = trained

In [17]:
# model.summary()

In [18]:
# note_events[:10]

### Convert chord names to MIDI pitch information

We need some way to convert e.g. "Eb-" to useful note data. `music21` looks handy for that (on second thought, perhaps I should rewrite MIDI functions to use `music21` as well):

In [19]:
[pitch.midi for pitch in music21.harmony.ChordSymbol("E-m").pitches]

[51, 54, 58]

Alright, so we have a bit of conversion to do to get it in a nice format for `music21.harmony.ChordSymbol` (e.g. from "Eb-" to "E-m"). This takes some string wrangling:

In [20]:
def db_chord_to_music21_chord(chord):
    new_chord = chord
    
    # Use "m" instead of "-" to indicate minor chords
    new_chord = new_chord.replace("-", "m")
    
    # Use "maj" instead of "j" to indicate maj7 chords
    new_chord = new_chord.replace("j", "maj")
    
    # Use "sus4add7" instead of "sus7" for suspended 7th chords
    # TODO: Get it to add a dominant 7th ("sus4add7" adds a major 7th)
    #       (probably could use music21.harmony.addNewChordSymbol())
    new_chord = new_chord.replace("sus7", "sus4")
    
    # According to music21 docs: "if a root or bass is flat, the ‘-‘ must be used, and NOT ‘b’.
    # However, alterations and chord abbreviations are specified normally with the ‘b’ and ‘#’ signs."
    if len(new_chord) > 1 and new_chord[1] == "b":
        new_chord_arr = list(new_chord)
        new_chord_arr[1] = "-"
        new_chord = "".join(new_chord_arr)
    
    # Convert e.g. "B-79b" to "B-7b9"
    if len(new_chord) > 0 and new_chord[-1] == "b":
        new_chord_arr = list(new_chord)
        if len(new_chord_arr) > 2:
            new_chord_arr[-1] = new_chord_arr[-2]
            new_chord_arr[-2] = "b"
        new_chord = "".join(new_chord_arr)
        
    return new_chord

chord_set = set(chords)
chord_set_converted = set([db_chord_to_music21_chord(chord) for chord in chord_set])
chord_set_converted
[music21.harmony.ChordSymbol(chord) for chord in chord_set_converted if chord != "NC"][:10]

# for chord in chord_set_converted:
#     print(chord)
#     if chord != "NC":
#         print(music21.harmony.ChordSymbol(chord))

[<music21.harmony.ChordSymbol G-7>,
 <music21.harmony.ChordSymbol Am7>,
 <music21.harmony.ChordSymbol Gmaj7>,
 <music21.harmony.ChordSymbol B-7>,
 <music21.harmony.ChordSymbol Fm7>,
 <music21.harmony.ChordSymbol D-m7>,
 <music21.harmony.ChordSymbol D-7>,
 <music21.harmony.ChordSymbol D7>,
 <music21.harmony.ChordSymbol C-maj7>,
 <music21.harmony.ChordSymbol E-maj7>]

And then to MIDI pitches:

In [21]:
def db_chord_to_midi_pitches(chord):  # TODO: Add optional octave argument
    if chord == "NC":
        return []
    chord_name = db_chord_to_music21_chord(chord)
    chord_symbol = music21.harmony.ChordSymbol(chord_name)
    return [pitch.midi for pitch in chord_symbol.pitches]

[db_chord_to_midi_pitches(chord) for chord in chord_set][:10]

[[49, 52, 56, 59],
 [42, 46, 49, 52],
 [49, 53, 56, 59],
 [45, 48, 52, 55],
 [47, 51, 54, 58],
 [41, 44, 48, 51],
 [],
 [51, 55, 58, 62],
 [46, 50, 53, 56],
 [50, 54, 57, 60]]

OK I think we're ready to start generating some notes!

### Generate notes

We'll have to rewrite the function that retrieves keys for note events, so that it will handle chord info:

In [22]:
def midi_get_note_event_keys(mode="single_melody"):
    """Get keys for note events."""
    if mode == "single_melody":
        # Don't use note off velocity to shrink possibilities,
        # and don't use note off pitch because it's the same as note on pitch
        return ("noteon_pitch", "noteon_velocity",
                "noteon_time", "noteoff_time")
    elif mode == "single_melody_harmony":
        return ("noteon_pitch", "noteon_velocity",
                "noteon_time", "noteoff_time",
                "chord")
    return ()

midi_get_note_event_keys(mode="single_melody_harmony")

('noteon_pitch', 'noteon_velocity', 'noteon_time', 'noteoff_time', 'chord')

All we have to do to `lstm_generate_midi_output` is pass the mode into `midi_write_file`:

In [23]:
def lstm_generate_midi_output(model, note_events,
                              mode="single_melody",
                              num_notes_to_generate=100,
                              batch_size=32,
                              random_seed=False,
                              add_seed_to_output=False,
                              output_filepath=None,
                              midi_source_filepath=None,
                              tune_name="output",
                              data_dir="../data/output/"):
    """Generate note output, given a trained model."""
    # Construct input sequence
    seq_in = lstm_construct_input_seq(note_events, model.input_shape[1],
                                      random_seed=random_seed)
    print("Constructed input sequence: {}".format(seq_in))

    # Generate the notes!
    num_notes = num_notes_to_generate
    notes_out = lstm_generate_notes(model, note_events, seq_in,
                                    num_notes_to_generate=num_notes,
                                    batch_size=batch_size,
                                    add_seed_to_output=add_seed_to_output)
    print("Generated {} notes".format(num_notes))

    # Write output to MIDI file
    if output_filepath is None:
        output_filepath = midi_construct_output_filepath(tune_name, data_dir)
    midi_write_file(notes_out, output_filepath,
                    mode=mode,
                    midi_source_filepath=midi_source_filepath)
    print("Wrote to MIDI file at {}".format(output_filepath))
    return (notes_out, output_filepath)

Similarly, get `midi_write_file` to pass the mode into `midi_get_note_event_keys`, and also write the chord information to file:

In [35]:
def midi_write_file(note_events, output_filepath,
                    mode="single_melody",
                    midi_source_filepath=None):
    """Write note events to MIDI file.

    Convert the sequence of note tuples into a sequence of MIDI notes,
    and then write to MIDI file.
    """
    # Create MIDI file and track
    midi_file_out = mido.MidiFile()
    midi_track_out = mido.MidiTrack()
    midi_file_out.tracks.append(midi_track_out)

    # Append "headers" (track name, tempo, key, time signature)
    if midi_source_filepath is not None:
        midi_track = midi_load_melody_from_file(midi_source_filepath)
        for message in midi_track[:4]:
            midi_track_out.append(message)
    else:
        pass

    # Add notes
    prev_time = 0
    note_events_keys = midi_get_note_event_keys(mode=mode)

    # Note times get all bunched together, so we stretch them out
    # a little bit manually here...
    # TODO: Revisit this to make it more robust
    time_multiplier = 2  # Art Pepper - Anthropology
    time_multiplier = 0.02  # Coleman Hawkins - Body and Soul
    time_multiplier = 2.5  # John Coltrane - Giant Steps
    
    # Harmony (chord) settings
    prev_chord = "NC"
    chord_velocity = 64

    for note in note_events:
        # Note on/off pairs
        note = dict((note_events_keys[i], note[i]) for i, _ in enumerate(note))
        noteon_time = int(note["noteon_time"] * time_multiplier)
        noteoff_time = int(note["noteoff_time"] * time_multiplier)
        curr_time_noteon = prev_time + noteon_time
        curr_time_noteoff = prev_time + noteoff_time
        # prev_time = curr_time_noteoff
        message_noteon = mido.Message("note_on",
                                      note=note["noteon_pitch"],
                                      velocity=note["noteon_velocity"],
                                      time=curr_time_noteon)
        message_noteoff = mido.Message("note_off",
                                       note=note["noteon_pitch"],
                                       velocity=note["noteon_velocity"],
                                       time=curr_time_noteoff)
        midi_track_out.append(message_noteon)
        midi_track_out.append(message_noteoff)
        
        # Write harmony (chords) as well
        if mode == "single_melody_harmony":
            curr_chord = note["chord"]
            if curr_chord != prev_chord:
                curr_pitches = db_chord_to_midi_pitches(curr_chord)
                prev_pitches = db_chord_to_midi_pitches(prev_chord)
                
                # Add note ons for current chord
                for pitch in curr_pitches:
                    message = mido.Message("note_on",
                                           note=pitch,
                                           velocity=chord_velocity,
                                           time=0)  # time=curr_noteon
                    midi_track_out.append(message)

                # Add note offs for previous chord
                for pitch in prev_pitches:
                    message = mido.Message("note_off",
                                           note=pitch,
                                           velocity=chord_velocity,
                                           time=0)  # time=curr_noteon
                    midi_track_out.append(message)
                
            prev_chord = curr_chord

    # Save file to disk
    midi_file_out.save(output_filepath)

    # for message in midi_track_out[4:20]:
    #     print(message)
    return output_filepath

### Putting it all together

Train the model:

In [25]:
trained = lstm_train_on_midi_input(tune_name=tune_name,
                                   mode="single_melody_harmony",
                                   midi_data_dir="../data/midi/quantized/",
                                   db_beats_filepath=beats_filepath,
                                   db_melody_filepath=melody_filepath,
                                   seq_length=10,
                                   num_epochs=1)
model, note_events, input_filepath = trained

/Users/usdivad/.pyenv/versions/3.5.1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


Created 1127 note events from ../data/midi/quantized/JohnColtrane_GiantSteps-1_FINAL.mid using mode single_melody_harmony
Formatted note data (1117 seqs of length 10, 601 unique notes)
Created model
No weights loaded (to load weights, specify a `weights_filepath`)
Epoch 1/1
1117/1117 [==============================] - 6s - loss: 6.3668     
Trained model over 1 epochs


Load weights ay:

In [26]:
# Coleman Hawkins - Body and Soul

# 1 epoch
# lstm_load_weights(model, "../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_00_6.3755.hdf5")

# 10 epochs
# lstm_load_weights(model, "../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_09_5.4847.hdf5")

# 50 epochs
# lstm_load_weights(model, "../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_49_1.5432.hdf5")

# 75 epochs
# lstm_load_weights(model,"../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_74_0.8663.hdf5")

# 100 epochs
# lstm_load_weights(model,"../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703162618875647_99_0.4374.hdf5")

# 150 epochs
# lstm_load_weights(model,"../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703165925088780_146_0.2167.hdf5")

# 200 epochs
# lstm_load_weights(model,"../data/models/weights_ColemanHawkins_BodyAndSoul_FINAL_20170703165925088780_170_0.1458.hdf5")

Use the model to generate some notes:

In [39]:
notes_out, output_filepath = lstm_generate_midi_output(model, note_events,
                                                       mode="single_melody_harmony",
                                                       num_notes_to_generate=100,
                                                       random_seed=False,
                                                       add_seed_to_output=True,
                                                       tune_name=tune_name,
                                                       midi_source_filepath=input_filepath,
                                                       data_dir="../data/output")
notes_out[:20]

Constructed input sequence: [154, 254, 395, 529, 475, 395, 313, 212, 106, 186]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_JohnColtrane_GiantSteps-1_FINAL_20170703181936832548.mid


[(61, 110, 0, 96, 'NC'),
 (64, 110, 0, 96, 'NC'),
 (68, 110, 0, 96, 'NC'),
 (71, 120, 0, 96, 'NC'),
 (70, 110, 0, 288, 'NC'),
 (68, 110, 0, 96, 'NC'),
 (66, 110, 0, 96, 'Cbj7'),
 (63, 110, 0, 96, 'Cbj7'),
 (59, 100, 0, 240, 'Cbj7'),
 (62, 110, 0, 96, 'D7'),
 (64, 110, 0, 48, 'D7'),
 (66, 100, 0, 96, 'D7'),
 (69, 110, 0, 96, 'D7'),
 (67, 110, 0, 96, 'Gj7'),
 (62, 100, 0, 96, 'Gj7'),
 (59, 100, 0, 96, 'Gj7'),
 (59, 100, 0, 96, 'Bb7'),
 (56, 110, 0, 96, 'Bb7'),
 (55, 100, 0, 96, 'Bb7'),
 (53, 100, 0, 96, 'Bb7')]

In [28]:
# Trying to do MIDI -> mp3 conversion directly in notebook
# %%bash -s "$output_filepath"
# source ~/.bash_profile
# miditomp3 $1

In [29]:
# Play the MIDI file here
# music21.converter.parse("../data/output/out_ColemanHawkins_BodyAndSoul_FINAL_20170703163206182499.mid").show("midi")

Load multiple weights and generate output for each of them (so we can compare):

In [44]:
weights_filepaths = ["weights_JohnColtrane_GiantSteps-1_FINAL_20170703180209718872_00_6.3668.hdf5",
                     "weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_09_5.7838.hdf5",
                     "weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_49_2.0546.hdf5",
                     "weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_99_0.7281.hdf5"]

weights_filepaths = [os.path.join("../data/models/", path) for path in weights_filepaths]

for weights_filepath in weights_filepaths:
    lstm_load_weights(model, weights_filepath)
    print("Loaded weights from {}".format(weights_filepath))

    output = lstm_generate_midi_output(model, note_events,
                                       mode="single_melody_harmony",
                                       num_notes_to_generate=100,
                                       random_seed=False,
                                       add_seed_to_output=True,
                                       tune_name=tune_name,
                                       midi_source_filepath=input_filepath,
                                       data_dir="../data/output")

Loaded weights from ../data/models/weights_JohnColtrane_GiantSteps-1_FINAL_20170703180209718872_00_6.3668.hdf5
Constructed input sequence: [154, 254, 395, 529, 475, 395, 313, 212, 106, 186]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_JohnColtrane_GiantSteps-1_FINAL_20170703182446619918.mid
Loaded weights from ../data/models/weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_09_5.7838.hdf5
Constructed input sequence: [154, 254, 395, 529, 475, 395, 313, 212, 106, 186]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_JohnColtrane_GiantSteps-1_FINAL_20170703182448252889.mid
Loaded weights from ../data/models/weights_JohnColtrane_GiantSteps-1_FINAL_20170703181128071625_49_2.0546.hdf5
Constructed input sequence: [154, 254, 395, 529, 475, 395, 313, 212, 106, 186]
Generated 100 notes
Wrote to MIDI file at ../data/output/out_JohnColtrane_GiantSteps-1_FINAL_20170703182450020260.mid
Loaded weights from ../data/models/weights_JohnColtrane_GiantSteps-1_FINAL_2

(

Run this whenever we want to train the model even further (but don't forget to update the `weights_filepath`):

In [38]:
# trained = lstm_train_on_midi_input(tune_name=tune_name,
#                                    mode="single_melody_harmony",
#                                    midi_data_dir="../data/midi/quantized/",
#                                    db_beats_filepath=beats_filepath,
#                                    db_melody_filepath=melody_filepath,
#                                    weights_filepath=None,
#                                    seq_length=10,
#                                    num_epochs=100)
# model, note_events, input_filepath = trained

)